In [1]:
import os
import re

from collections import Counter
from copy import copy

import numpy as np

from tqdm import tqdm

In [2]:
SEED = 42
np.random.seed(SEED)

In [3]:
DATA_PATH = './corpora'

with open(os.path.join(DATA_PATH, 'AnnaKarenina.txt'), 'r') as f1, \
     open(os.path.join(DATA_PATH, 'WarAndPeace.txt'), 'r') as f2:
    anna_karenina = f1.read().lower()
    war_and_peace = f2.read().lower()

In [4]:
ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

### 1 - Униграммы

Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [5]:
def filter_text(text):
    return ''.join(l for l in text.lower() if l in ALPHABET)

def get_unigram_freqs(text):
    freqs = Counter(text)
    freqs = {k: v / sum(freqs.values()) for k, v in freqs.items()}
    freqs = dict(sorted(freqs.items(), key=lambda x: x[1], reverse=True))
    return freqs

def unigram_encode_decode(text, mapping):
    return ''.join([mapping[l] for l in text])

def get_encode_mapping(orig_freqs):
    return dict(zip(orig_freqs, np.random.permutation(list(orig_freqs))))

def get_decode_mapping(orig_freqs, text_freqs):
    return dict(zip(text_freqs, orig_freqs))

def accuracy(orig_text, decoded_text):
    return sum(np.array(list(orig_text)) == np.array(list(decoded_text))) / len(orig_text)

In [6]:
train_texts = filter_text(anna_karenina + war_and_peace)

unigram_freqs = get_unigram_freqs(train_texts)
encode_mapping = get_encode_mapping(unigram_freqs)

In [7]:
test_text = """
Да, но не знать о Солнечной системе!.. – воскликнул я. 
– На кой черт она мне? – перебил он нетерпеливо. 
– Ну хорошо, пусть, как вы говорите, мы вращаемся вокруг Солнца. 
А если бы я узнал, что мы вращаемся вокруг Луны, 
много бы это помогло мне или моей работе? Я хотел было спросить, что же это за работа, но почувствовал, что он будет недоволен. 
Я задумался над нашим коротким разговором и попытался сделать кое-какие выводы. 
Он не хочет засорять голову знаниями, которые не нужны для его целей. 
Стало быть, все накопленные знания он намерен так или иначе использовать. 
Я перечислил в уме все области знаний, в которых он проявил отличную осведомленность. 
Я даже взял карандаш и записал все это на бумаге. 
Перечитав список, я не мог удержаться от улыбки. 
"""

In [8]:
filtered_text = filter_text(test_text)
encoded_text = unigram_encode_decode(filtered_text, encode_mapping)

In [9]:
text_unigram_freqs = get_unigram_freqs(encoded_text)
decode_mapping = get_decode_mapping(unigram_freqs, text_unigram_freqs)

decoded_text = unigram_encode_decode(encoded_text, decode_mapping)

In [10]:
print(filtered_text)

да но не знать о солнечной системе  воскликнул я  на кой черт она мне  перебил он нетерпеливо  ну хорошо пусть как вы говорите мы вращаемся вокруг солнца а если бы я узнал что мы вращаемся вокруг луны много бы это помогло мне или моей работе я хотел было спросить что же это за работа но почувствовал что он будет недоволен я задумался над нашим коротким разговором и попытался сделать коекакие выводы он не хочет засорять голову знаниями которые не нужны для его целей стало быть все накопленные знания он намерен так или иначе использовать я перечислил в уме все области знаний в которых он проявил отличную осведомленность я даже взял карандаш и записал все это на бумаге перечитав список я не мог удержаться от улыбки 


In [11]:
print(decoded_text)

ьн ао ае ганиз о лотаеыаож лслиеке  волдтсдаут м  ан дож ыери оан кае  яеребст оа аеиеряетсво  ау йорохо яулиз днд вп човорсие кп врнэнеклм водруч лотацн н елтс бп м угант ыио кп врнэнеклм водруч туап каочо бп юио яокочто кае стс коеж рнбоие м йоиет бпто ляролсиз ыио ше юио гн рнбоин ао яоыувливовнт ыио оа буьеи аеьовотеа м гньукнтлм ань анхск дороидск рнгчоворок с яояпинтлм льетниз доедндсе впвоьп оа ае йоыеи гнлормиз чотову ганасмкс доиорпе ае аушап ьтм ечо цетеж линто бпиз вле андоятеаапе ганасм оа анкереа инд стс саные сляотзговниз м яереыслтст в уке вле обтнлис ганасж в доиорпй оа яромвст оитсыаущ олвеьоктеааолиз м ьнше вгмт днрнаьнх с гняслнт вле юио ан букнче яереысинв ляслод м ае коч уьершнизлм ои утпбдс 


In [12]:
print(f'Доля верно расшифрованных символов: {accuracy(filtered_text, decoded_text):.4f}')

Доля верно расшифрованных символов: 0.4626


Вроде как доля верно расшифрованных символов нормальная, но прочитать текст не представляется возможным.

### 2 - Биграммы

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [13]:
def get_bigram_freqs(text):
    freqs = Counter([text[i:i+2] for i in range(len(text) - 2)])
    freqs = {k: v / sum(freqs.values()) for k, v in freqs.items()}
    freqs = dict(sorted(freqs.items(), key=lambda x: x[1], reverse=True))
    return freqs

def bigram_decode(text, mapping):
    UNKNOWN = '?'
    decoded_text = [UNKNOWN] * len(text) 
    for text_bi, orig_bi in mapping.items(): # сначала заполняем самые частотные
        index = text.find(text_bi)
        while index != -1:
            if decoded_text[index] == UNKNOWN:
                decoded_text[index] = orig_bi[0]
            if decoded_text[index + 1] == UNKNOWN:
                decoded_text[index + 1] = orig_bi[1] 
            index = text.find(text_bi, index + 1)

    return ''.join(decoded_text)

In [14]:
bigram_freqs = get_bigram_freqs(train_texts)

In [15]:
text_bigram_freqs = get_bigram_freqs(encoded_text)
decode_mapping = get_decode_mapping(bigram_freqs, text_bigram_freqs)

decoded_text = bigram_decode(encoded_text, decode_mapping)

In [16]:
print(f'Доля верно расшифрованных символов: {accuracy(filtered_text, decoded_text):.4f}')

Доля верно расшифрованных символов: 0.1233


In [17]:
print(decoded_text)

и и аи о а сто а о б пем оноул р о  но ягоя мя е и сио отионв в сиа о ь онаноя вои ппонрего а и мак тчуа  ялтотота ннелр отапо ан нй ь т ое ноти  но бсп и иэкгоиннее ер ся иаа ан нй ь т ое ноти  зкдбнеам ла ннееаа  ме р а а о огокагвотй у ко е к кря нк а ос ч атотиаа то еаа а ий у к и а  мячал  оа я иаа вовнутвви плр об опе аелит доеи сли сдсдко т касдий тлр отчевнь меот дое отн етото ввтасо нв ойн вои о к етвнаее титотл браона сигишнко алтео  о  ммбневбе эла ьн  оно  да нкто нно  с  ек гбыо а сиге вои саонаовтьавогоио сео о о брсраетоте  онаеу гоя нтеао нно вшисл нна сиго нко алтег воь  чаноя вкгою мн в еслсек гм лтоте илто нюея оий иилдвннаеау  я нно еааи синитрхо  онаеатьоносу рдпеи о а  тетонретоое вкаеесхани


Стало еще хуже, но оно и понятно - количество биграмм сильно больше, чем букв, попасть сложнее. Думаю, можно лучше, если декодировать по-умному.

### 3  - MCMC

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

Алгоритм похож на генетический. Повторяем n_iter раз:
- Меняем местами две случайные буквы в ключе расшифровки
- Оцениваем вероятность получить такой текст. У нас есть наша "языковая модель", построенная на тренировочных текстах. Из нее мы знаем вероятности встретить биграммы. Таким образом, мы можем посчитать likelihood декодированного текста как произведение вероятностей для всех биграмм в нем.
- Принимаем или отклоняем новый ключ шифрования (если новый likelihood больше, то принимаем, если меньше, то принимаем с вероятностью new_llh / old_llh)

In [18]:
class MCMC:
    
    def __init__(self, known_freqs, alphabet, n_iter=20000, verbose=5000):
        self.n_iter = n_iter
        self.verbose = verbose
        self.known_freqs = known_freqs
        self.alphabet = alphabet
        
        self.nan_replacement = 1 / len(ALPHABET) ** 2
        
    def loglikelihood(self, text):
        bigram_counts = Counter([''.join(text[i: i + 2]) for i in range(len(text) - 2)])
        return np.sum([count * np.log(self.known_freqs.get(bigram, self.nan_replacement)) 
                       for bigram, count in bigram_counts.items()])
             
    def _mutate(self, text):
        letters = np.random.choice(self.alphabet, 2, replace=False)
        for i in range(len(text)):
            if text[i] == letters[0]:
                text[i] = letters[1]
            elif text[i] == letters[1]:
                text[i] = letters[0]
        return text
    
    @staticmethod
    def _accept(cur_llh, new_llh):
        if new_llh > cur_llh:
            return True
        return np.random.rand() < np.exp(new_llh - cur_llh)
        
    def decode(self, text):
        best_decoded_text = copy(text)
        cur_llh = best_llh = self.loglikelihood(text) 
        for iteration in tqdm(range(self.n_iter)):
            new_text = self._mutate(copy(text))
            new_llh = self.loglikelihood(new_text)
            if self._accept(cur_llh, new_llh):
                text = new_text
                cur_llh = new_llh
                if cur_llh > best_llh:
                    best_llh = cur_llh
                    best_decoded_text = copy(text)
                    
            if iteration % self.verbose == 0:
                print(f'Best LLH: {best_llh}')
                print(''.join(best_decoded_text))
                print('--------------------------------------------------------------------------------')
                
        return ''.join(best_decoded_text)

In [24]:
mcmc = MCMC(bigram_freqs, list(ALPHABET))

mcmc.decode(list(encoded_text))

  1%|          | 247/20000 [00:00<00:24, 811.73it/s]

Best LLH: -6274.3579092772
яхплыплъпцлхзопыпдыюлъжлычпдйдзъьъппвыднюйнлиюпкпплхпнычпжъ зпылхпьлъппмъ ъайюпылплъзъ мъюйвыпплипшы ыгыпмидзопнхнпвщпеывы йзъпьщпв хсхъьдкпвын иепдыюлёхпхпъдюйпащпкпицлхюпжзыпьщпв хсхъьдкпвын иепюилщпьлыеыпащпбзыпмыьыеюыпьлъпйюйпьыъчп хаызъпкпшызъюпащюыпдм ыдйзопжзыпфъпбзыпцхп хаызхплыпмыживдзвывхюпжзыпылпаияъзплъяывыюълпкпцхяиьхюдкплхяплхгйьпны ызнйьп хцеывы ыьпйпмымщзхюдкпдяъюхзопныънхнйъпвщвыящпылплъпшыжъзпцхды кзопеыюывипцлхлйкьйпнызы щъплъплифлщпяюкпъеыпёъюъчпдзхюыпащзопвдъплхнымюъллщъпцлхлйкпылплхьъ ълпзхнпйюйпйлхжъпйдмыюоцывхзопкпмъ ъжйдюйюпвпиьъпвдъпыаюхдзйпцлхлйчпвпнызы щшпылпм ыквйюпызюйжлитпыдвъяыьюъллыдзопкпяхфъпвцкюпнх хляхгпйпцхмйдхюпвдъпбзыплхпаиьхеъпмъ ъжйзхвпдмйдынпкплъпьыепияъ фхзодкпызпиющанйп
--------------------------------------------------------------------------------


 26%|██▌       | 5176/20000 [00:06<00:17, 841.88it/s]

Best LLH: -3945.3524457172102
да но не знать о солнечной системе  воскликнул я  на кой черт она мне  перебил он нетерпеливо  ну гороёо пусть как вы ховорите мы враъаемся вокрух солнца а если бы я узнал что мы враъаемся вокрух луны мнохо бы это помохло мне или моей работе я готел было спросить что же это за работа но почувствовал что он будет недоволен я задумался над наёим коротким разховором и попытался сделать коекакие выводы он не гочет засорять холову знаниями которые не нужны для ехо целей стало быть все накопленные знания он намерен так или иначе использовать я перечислил в уме все области знаний в которыг он проявил отличную осведомленность я даже взял карандаё и записал все это на бумахе перечитав список я не мох удержаться от улыбки 
--------------------------------------------------------------------------------


 51%|█████     | 10194/20000 [00:12<00:11, 844.05it/s]

Best LLH: -3928.435222215997
да но не знать о солнечной системе  воскликнул я  на кой черт она мне  перебил он нетерпеливо  ну хороёо пусть как вы говорите мы враъаемся вокруг солнца а если бы я узнал что мы враъаемся вокруг луны много бы это помогло мне или моей работе я хотел было спросить что же это за работа но почувствовал что он будет недоволен я задумался над наёим коротким разговором и попытался сделать коекакие выводы он не хочет засорять голову знаниями которые не нужны для его целей стало быть все накопленные знания он намерен так или иначе использовать я перечислил в уме все области знаний в которых он проявил отличную осведомленность я даже взял карандаё и записал все это на бумаге перечитав список я не мог удержаться от улыбки 
--------------------------------------------------------------------------------


 76%|███████▌  | 15210/20000 [00:18<00:05, 844.42it/s]

Best LLH: -3928.435222215997
да но не знать о солнечной системе  воскликнул я  на кой черт она мне  перебил он нетерпеливо  ну хороёо пусть как вы говорите мы враъаемся вокруг солнца а если бы я узнал что мы враъаемся вокруг луны много бы это помогло мне или моей работе я хотел было спросить что же это за работа но почувствовал что он будет недоволен я задумался над наёим коротким разговором и попытался сделать коекакие выводы он не хочет засорять голову знаниями которые не нужны для его целей стало быть все накопленные знания он намерен так или иначе использовать я перечислил в уме все области знаний в которых он проявил отличную осведомленность я даже взял карандаё и записал все это на бумаге перечитав список я не мог удержаться от улыбки 
--------------------------------------------------------------------------------


100%|██████████| 20000/20000 [00:23<00:00, 842.18it/s]


'да но не знать о солнечной системе  воскликнул я  на кой черт она мне  перебил он нетерпеливо  ну хороёо пусть как вы говорите мы враъаемся вокруг солнца а если бы я узнал что мы враъаемся вокруг луны много бы это помогло мне или моей работе я хотел было спросить что же это за работа но почувствовал что он будет недоволен я задумался над наёим коротким разговором и попытался сделать коекакие выводы он не хочет засорять голову знаниями которые не нужны для его целей стало быть все накопленные знания он намерен так или иначе использовать я перечислил в уме все области знаний в которых он проявил отличную осведомленность я даже взял карандаё и записал все это на бумаге перечитав список я не мог удержаться от улыбки '

Считаю, что удалось расшифровать

## 4 - Расшифровка сообщения

In [25]:
to_decode = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [26]:
to_decode_unigram_freqs = get_unigram_freqs(to_decode)
to_decode_mapping = get_decode_mapping(unigram_freqs, to_decode_unigram_freqs)

to_decode_text = unigram_encode_decode(to_decode, to_decode_mapping)

In [27]:
print(to_decode_text)

олна рд раяасо иевьтнуидг ана пемса иевьтнуидг соклс б шсеые леезжоиач кесевдг ноыке пвемастсу лкевоо рлоые рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуидг зтнн йт пелнояиоо мосровсео йтятиао кбвлт хесч кеиомие ч иамоые ио езожтю


In [29]:
mcmc = MCMC(bigram_freqs, list(ALPHABET), n_iter=50000, verbose=10000)

mcmc.decode(list(to_decode_text))

  1%|          | 568/50000 [00:00<00:26, 1861.12it/s]

Best LLH: -1547.4474736920654
олна рд раяасо иевьтнуидг ана пемса иевьтнуидг соклс б шсеые леезжоиач кесевдг ноыке пвемастсу лкевоо рлоые рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуидг зтнн йт пелнояиоо мосровсео йтятиао кбвлт хесч кеиомие ч иамоые ио езожтю
--------------------------------------------------------------------------------


 21%|██        | 10564/50000 [00:05<00:19, 1984.14it/s]

Best LLH: -1233.8237303205765
если вы вимите нордальный или почти нордальный текст у этого соожёения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожеёаш
--------------------------------------------------------------------------------


 41%|████      | 20530/50000 [00:10<00:14, 1980.27it/s]

...

Best LLH: -1232.7373345243193
если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешах
--------------------------------------------------------------------------------


 81%|████████  | 40593/50000 [00:20<00:04, 1986.45it/s]

Best LLH: -1232.7373345243193
если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешах
--------------------------------------------------------------------------------


100%|██████████| 50000/50000 [00:25<00:00, 1964.64it/s]


'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешах'

Текст прочитать можно=) считаю, что расшифровка удалась